# Slice Builder: GraphML Parsing

## Pick a Slice Name, Site to Work on, and the GraphML File to Parse

In [ ]:
# Constants to change as needed
SLICE_NAME = "Clos"
SITE_NAME = "MASS"
GRAPH_PATH = "graphs/2tier_test.graphml"

## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try: 
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Parse GraphML for Topology and Create Slice

In [ ]:
import xml.dom.minidom
from collections import Counter

try:        
    #Create the slice
    slice = fablib.new_slice(name=SLICE_NAME)
    
    # Create dictionary to store nodes
    nodeDict = {}
    
    # Use XML parser to parse the GraphML file
    docs = xml.dom.minidom.parse(GRAPH_PATH)

    # Find all nodes via the node tag, add each to the slice with Rocky Linux as its base
    nodes = docs.getElementsByTagName("node")
    for node in nodes:
        nodeName = node.getAttribute("id")
        nodeInfo = slice.add_node(name=nodeName, cores=1, ram=4, image='default_rocky_8', site=SITE_NAME)
        nodeDict[nodeName] = [nodeInfo, 1]
        
        print(f'Added node {nodeName}')
    
    # Find all edges via the edge tag, add each to the slice via an L2Bridge connecting the node interfaces
    edges = docs.getElementsByTagName("edge")
    for edge in edges:
        source = edge.getAttribute("source")
        target = edge.getAttribute("target")
        
        sourceIntfName = "{node}_eth{num}".format(node=source, num=nodeDict[source][1])
        targetIntfName = "{node}_eth{num}".format(node=target, num=nodeDict[target][1])
        networkName = f'link_{source}_{target}'
        
        # Add a NIC for each node that is a part of the edge
        sourceIntf = nodeDict[source][0].add_component(model='NIC_Basic', name=sourceIntfName).get_interfaces()[0]
        targetIntf = nodeDict[target][0].add_component(model='NIC_Basic', name=targetIntfName).get_interfaces()[0]
        
        nodeDict[source][1] += 1
        nodeDict[target][1] += 1

        # Add a L2 network between the interfaces
        slice.add_l2network(name=networkName, interfaces=[sourceIntf, targetIntf], type="L2Bridge")
        
        print(f'Added edge {source}-{target}')

     
    #Submit Slice Request
    slice.submit()

except Exception as e:
    print(f"Exception: {e}")